## Interactive Plots Integration (Plotly, Bokeh, Highcharts)

PyBloqs also supports plotting with interactive plotting libraries such as Plotly(offline), Bokeh and Highcharts.

The plots will be rendered in HTML and will rely on your browser's CSS and JS capabilities to provide interactivity.

### Plotly Example

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("country=='Canada'")
plotly_fig = px.line(df, x="year", y="lifeExp", title="Life expectancy in Canada")
plotly_fig

In [ ]:
import pybloqs as p

p.Block(plotly_fig)

### Bokeh Example

In [ ]:
import numpy as np

# The lines below are only needed for notebook output
from bokeh.io.output import output_notebook
from bokeh.plotting import figure as b_fig
from bokeh.resources import INLINE

output_notebook(resources=INLINE)

In [ ]:
N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors2 = [f"#{int(r):02x}{int(g):02x}{150:02x}" for r, g in zip(50 + 2 * x, 30 + 2 * y)]
bokeh_fig = b_fig(width=300, height=300)
bokeh_fig.scatter(x, y, radius=radii, fill_color=colors2, fill_alpha=0.6, line_color=None)

In [ ]:
p.Block(bokeh_fig)

### Combining Bokeh and Plotly plots with HStack

In [ ]:
p.HStack([p.Block(bokeh_fig), p.Block(plotly_fig)])

### Highcharts examples
Please note: Highcharts has a [proprietary license](https://shop.highsoft.com/highcharts-t2)

#### Simple line chart
When evaluated as the last expression in a Notebook Cell, the plot is automatically displayed inline.
Note how the plot name (hover over the line to see the little popup) is taken from the input data (if available).

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd

df = pd.DataFrame(
    (np.random.rand(200, 4) - 0.5) / 10, columns=list("ABCD"), index=pd.date_range(datetime(2000, 1, 1), periods=200)
)
df_cr = (df + 1).cumprod()
a = df_cr["A"]
b = df_cr["B"]
c = df_cr["C"]

In [ ]:
import pybloqs.plot as pp

pp.interactive()
pp.Plot(a)

### Saving as interactive HTML

In [ ]:
pp.Plot(df).save("chart_sample.html")

#### Scatter Plot
Regular scatter plot, with zooming on both the x and y axes.

In [ ]:
pp.Plot(df.values[:, :2], pp.Scatter(pp.Marker(enabled=True)), pp.Chart(zoom_type="xy"))

#### Bar Charts 
Notice how when viewing all the data at once, the chart shows monthly data, yet zooming in reveals additional detail at up to daily resolution. This is accomplished by using a custom data grouping.

In [ ]:
bar_grouping = pp.DataGrouping(approximation="open", enabled=True, group_pixel_width=100)

In [ ]:
# Bar chart from a dataframe
pp.Plot(df, pp.Column(bar_grouping))

In [ ]:
# Stacked bar chart
pp.Plot(df, pp.Column(bar_grouping, stacking="normal"))

In [ ]:
# Composite bar chart from two separate plots.
s2 = pp.Plot([pp.Plot(a, pp.Column(bar_grouping)), pp.Plot(b, pp.Column(bar_grouping))])
s2

#### Comparing series in a dataframe

Plot the cumulative percentage difference between input series (or columns of a dataframe). The cumulative difference is always calculated from the start of the observation window. This results in intuitively correct behavior when zooming in or sliding the window, as the chart will dynamically recalculate the values. Incredibly useful for comparing model performance over time for example as one doesn't need to manually normalize money curves for different periods.


In [ ]:
s3 = pp.Plot(df_cr, pp.PlotOptions(pp.Series(compare="percent")), pp.TooltipPct(), pp.YAxisPct())
s3

#### Three series on separate side-by-side Y axes

In [ ]:
s4 = pp.Plot(
    [
        pp.Plot(a),
        pp.Plot(b, pp.YAxis(pp.Title(text="B Axis"), opposite=True)),
        pp.Plot(c, pp.YAxis(pp.Title(text="C Axis"), opposite=True, offset=40)),
    ]
)
s4

#### Two series on separate subplots

In [ ]:
s5 = pp.Plot(
    [
        pp.Plot(a, pp.Line(), pp.YAxis(pp.Title(text="a only"), height=150)),
        pp.Plot(b, pp.Column(), pp.YAxis(pp.Title(text="b only"), top=200, height=100, offset=0)),
    ],
    pp.Tooltip(value_decimals=2),
    height="400px",
)
s5

#### Creating a report from multiple charts and saving as HTML or PDF. Or sending it as email.

In [ ]:
import pandas.util.testing as pt

b = p.Block(
    [
        p.Block(pt.makeTimeDataFrame().tail(10), title="A table", title_level=1),
        p.Block([s2, s3], title="Side-by-side Plots", cols=2),
        p.Block(title="Full Width Plots", styles={"page-break-before": "always"}),
        p.Block(s4, title="Side by Side Axes"),
        p.Block(s5, title="Composite Plots"),
    ],
    title="Dynamic Reports",
)

In [ ]:
b.save("charts_test.pdf")
b.save("charts_test.html")